In [1]:
import requests
import pandas as pd
import numpy as np
from elasticsearch import Elasticsearch

In [2]:
ls

 CKAN_to_elastic_encrypted.ipynb    Untitled1.ipynb
 JUNAR_to_elastic_encrypted.ipynb   Untitled.ipynb
'kibana_migrate_index (1).ipynb'


In [3]:
url_ckan='http://datos.gob/api/3/action/package_list'
url_package='http://datos.gob/api/3/action/package_show?id='
url_package_search='http://datos.gob/api/3/action/package_search?'

In [4]:
#Listas de paquetes ckan (con sus ID)
ckan_packages=requests.get(url_ckan) #consultar api
ckan_packages=ckan_packages.json() #convertir respuesta a json

In [5]:
#OBTENER TODOS LOS PAQUETES DE LA BASE DE DATOS CKAN
rows=str(1000) #maximo numero de resultados por consulta (maximo permitido es 1000)
packs=[] #contenedor de paquetes retornados
for s in range(0,4):
    start=str(s*int(rows)) #paginacion
    packages=requests.get(url_package_search+'rows='+rows+'&'+'start='+start)
    packages=packages.json()
    packs.extend(packages['result']['results'])

In [6]:
recursos=[]
#recorrer recursos y ordenarlos en una lista (con ciertos atributos)
for p in packs:
    for r in p['resources']:
        d={} #contenedor para valores
        d['fecha']=r['last_modified'] #ultima modificacion
        d['formato']=r['format'] #formato
        if 'organization' in p.keys(): #si es que existe info de organizacion
            if p['organization']!=None:
                d['inst.']=p['organization']['title'] #si es que existe info del titulo incluirlo
            else:
                d['inst.']=None
        else:
            d['inst.']=None
        recursos.append(d)
recursos=pd.DataFrame(recursos) #convertir a df
recursos['fecha']=pd.to_datetime(recursos['fecha']) #convertir la fecha a formato solo con la fecha (sin la hora)
#desagregar en dia, mes, año
recursos['dia']=recursos['fecha'].dt.date
recursos['year']=recursos['fecha'].dt.year
recursos['mes']=recursos['fecha'].dt.month

In [7]:
data=recursos.to_dict(orient='records')

In [8]:
bulk_data=[]
for idx in range(0,len(data)):
    bulk_data.append({"index":{ "_index" : "ckan", "_type" : "dataset", "_id" : str(idx) }})
    bulk_data.append(data[idx])

In [9]:
#es = Elasticsearch(hosts = [ES_HOST],http_auth=('elastic', ''))

In [1]:
ES_HOST = "https://:9200"
INDEX_NAME = 'ckan'

In [2]:
from ssl import create_default_context

context = create_default_context(cafile="/certs/ca/ca.crt")
es = Elasticsearch(
    [ES_HOST],
    http_auth=('elastic',  ),
    scheme="https",
    port=443,
    ssl_context=context,
)

NameError: name 'Elasticsearch' is not defined

In [19]:
es.ping()

True

In [20]:
if es.indices.exists(INDEX_NAME):
    print("deleting '%s' index..." % (INDEX_NAME))
    res = es.indices.delete(index = INDEX_NAME)
    print(" response: '%s'" % (res))
# since we are running locally, use one shard and no replicas
request_body = {
    "settings" : {
        "number_of_shards": 1,
        "number_of_replicas": 0
    }
}
print("creating '%s' index..." % (INDEX_NAME))
res = es.indices.create(index = INDEX_NAME, body = request_body)
print(" response: '%s'" % (res))

deleting 'ckan' index...
 response: '{'acknowledged': True}'
creating 'ckan' index...
 response: '{'acknowledged': True, 'shards_acknowledged': True, 'index': 'ckan'}'


In [21]:
# bulk index the data
print("bulk indexing...")
res = es.bulk(index = INDEX_NAME, body = bulk_data, refresh = True)

bulk indexing...


In [24]:
# sanity check
res = es.search(index = INDEX_NAME, size=2, body={"query": {"match_all": {}}})
print(" response: '%s'" % (res))

 response: '{'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 9726, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'ckan', '_type': 'dataset', '_id': '0', '_score': 1.0, '_source': {'fecha': '2019-05-23T14:53:57.244939', 'formato': 'XLSX', 'inst.': 'Servicio Nacional de Aduanas', 'dia': '2019-05-23', 'year': 2019.0, 'mes': 5.0}}, {'_index': 'ckan', '_type': 'dataset', '_id': '1', '_score': 1.0, '_source': {'fecha': '2019-03-01T18:54:05.594707', 'formato': 'RAR', 'inst.': 'Servicio Nacional de Aduanas', 'dia': '2019-03-01', 'year': 2019.0, 'mes': 3.0}}]}}'


In [23]:
r = requests.get('https://es01:9200',auth=HTTPBasicAuth('elastic', ''),verify='/certs/ca/ca.crt')

NameError: name 'HTTPBasicAuth' is not defined

In [39]:
r.json()

{'name': 'es01',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'guh4RYBHSKKBTWpuee70vw',
 'version': {'number': '7.2.0',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '508c38a',
  'build_date': '2019-06-20T15:54:18.811730Z',
  'build_snapshot': False,
  'lucene_version': '8.0.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}